In [1]:
from src.utils import get_db_engine

engine = get_db_engine()

In [2]:
from src.utils import get_latest_record_date, get_db_engine

latest_date = get_latest_record_date(engine=engine)

In [3]:
latest_date.head()

,ticker,latest_record_date
0,A,2025-05-08
1,AA,2025-05-08
2,AAL,2025-05-08
3,AAON,2025-05-08
4,AAP,2025-05-08


In [4]:
latest_date.tail()

,ticker,latest_record_date
1493,ZD,2025-05-08
1494,ZI,2025-05-08
1495,ZION,2025-05-08
1496,ZTS,2025-05-08
1497,ZWS,2025-05-08


check if all stock records are up to the same date

In [5]:
assert (latest_date['latest_record_date'] == latest_date['latest_record_date'].max()).all()

In [ ]:
from sqlalchemy import select
from sqlalchemy.orm import Session

import pandas as pd 

from src.create_sql_tables import StockPrice


stmt = select(StockPrice.ticker.distinct().label('ticker'))

with Session(bind=engine) as session:
    result = session.execute(stmt)
    tickers = pd.DataFrame(result.fetchall(), columns=list(result.keys())).squeeze()

In [22]:
type(tickers)

pandas.core.series.Series

In [ ]:
all_tickers = tickers.to_list() # type:ignore

In [25]:
print(f'there are {len(all_tickers)} tickers')

there are 1498 tickers


let's do the fetching in 3 batch of around 500 tickers per batch.

In [26]:
tickers_1 = all_tickers[:500]
tickers_2 = all_tickers[500:1000]
tickers_3 = all_tickers[1000:]

assert len(tickers_1) + len(tickers_2) + len(tickers_3) == len(all_tickers)

In [31]:
last_record_date = latest_date['latest_record_date'].max()
last_record_date

datetime.date(2025, 5, 8)

In [32]:
last_record_date = str(last_record_date)
last_record_date

'2025-05-08'

In [33]:
from src.utils import download_from

df_1 = download_from(symbols=tickers_1, from_date=last_record_date)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  500 of 500 completed
/home/ping/Code/proto-collect-stock-data/src/utils.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Volume'] = df['Volume'].fillna(0).astype(int)


In [34]:
assert not df_1.isna().values.any()

In [35]:
df2 = download_from(symbols=tickers_2, from_date=last_record_date)

[*********************100%***********************]  500 of 500 completed


In [36]:
assert not df2.isna().values.any()

In [37]:
df3 = download_from(symbols=tickers_3, from_date=last_record_date)

[*********************100%***********************]  498 of 498 completed


In [38]:
assert not df3.isna().values.any()

In [39]:
df_1.head()

,trade_date,ticker,open_price,high_price,low_price,close_price,volume
0,2025-05-08,ALRM,53.71,55.74,53.53,55.31,542000
1,2025-05-08,AEIS,111.54,112.62,109.58,110.46,220300
2,2025-05-08,COHR,72.89,72.89,67.50,70.59,6311200
3,2025-05-08,CENTA,30.23,30.84,27.97,30.30,501900
4,2025-05-08,AMPH,24.59,25.36,23.68,24.65,751600


In [42]:
df2.head()

,trade_date,ticker,open_price,high_price,low_price,close_price,volume
0,2025-05-08,MUSA,489.79,496.30,439.51,443.56,1299900
1,2025-05-08,NOG,24.97,26.56,24.90,26.30,1881900
2,2025-05-08,FSLR,130.34,135.30,127.33,133.76,4269100
3,2025-05-08,LZB,40.85,42.31,40.85,42.27,303700
4,2025-05-08,JCI,90.81,92.41,90.35,91.24,9040800


In [43]:
df3.head()

,trade_date,ticker,open_price,high_price,low_price,close_price,volume
0,2025-05-08,TTGT,7.90,8.46,7.80,8.27,280100
1,2025-05-08,WEC,108.77,108.77,106.62,107.03,2423800
2,2025-05-08,TXT,70.62,73.07,70.52,72.22,1600800
3,2025-05-08,YELP,35.83,36.22,35.53,35.74,828200
4,2025-05-08,VOYA,65.60,66.91,65.01,66.44,845900


In [44]:
combined_df = pd.concat([df_1, df2, df3])

In [45]:
from src.create_sql_tables import StockPrice
from src.utils import insert_data

insert_data(data=combined_df.to_dict(orient='records'),  #type:ignore
            table=StockPrice)

In [50]:
#TODO: turn common statements into helper funcs

from sqlalchemy import select, func
from sqlalchemy.orm import Session

from src.create_sql_tables import StockPrice

# stmt = select(func.count()).select_from(my_table)
stmt = select(func.count()).select_from(StockPrice)

with Session(bind=engine) as session:
    result = session.execute(statement=stmt)
    print(f'there are {result.scalar_one():,} rows of stock price data.')

there are 10,770,972 rows of stock price data.
